Encoder Decoder stacks

In [1]:
print("Hello world")

Hello world


In [2]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
from torch.utils.data import DataLoader
import torchtext.datasets as datasets

c:\Users\tekju\Desktop\nlp_scratch\nlp_env\Lib\site-packages\torchtext\datasets\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\tekju\Desktop\nlp_scratch\nlp_env\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [3]:
def clones(module,N):
    "produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [4]:
class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
        
    def forward(self,x):
        mean = x.mean(-1,keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x-mean) / (std+self.eps) + self.b_2

In [5]:
ln = LayerNorm(512)

In [6]:
class SublayerConnection(nn.Module):
    "A residual connection followed by a layer norm."
    def __init__(self,size,dropout):
        super(SublayerConnection,self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x,sublayer):
        "apply residual conn to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x))) # pre-norm

In [7]:
class EncoderLayer(nn.Module):
    "self-attn and feedforward"
    def __init__(self,size, self_attn, feed_forward,dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size,dropout),2)
        self.size = size
        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [8]:
class DecoderLayer(nn.Module):
    
    def __init__(self,size,self_attn,src_attn, feed_forward, dropout):
        super().__init__()
        self.size= size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout),3)
        
    def forward(self,x,memory,src_mask,tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x,m,m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [9]:
class Encoder(nn.Module):
    "core encoder is a stack of N layers."
    def __init__(self, layer, N):
        super(Encoder,self).__init__()
        self.layers = clones(layer,N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self,x,mask):
        for layer in self.layers:
            x = layer(x,mask)
        return self.norm(x)

In [10]:
class Decoder(nn.Module):
    def __init__(self,layer,N):
        super(Decoder,self).__init__()
        self.layers = clones(layer,N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self,x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x,memory,src_mask,tgt_mask)
        return self.norm(x)

Attention and Mask

In [11]:
def subsequent_mask(size):
    "mask out subsequent positions."
    attn_shape = (1,size,size)
    subsequent_mask = torch.triu(torch.ones(attn_shape),diagonal=1).type(
    torch.uint8
    )
    return subsequent_mask==0

In [12]:
mask = subsequent_mask(3)
mask

tensor([[[ True, False, False],
         [ True,  True, False],
         [ True,  True,  True]]])

In [13]:
def attention(query, key, value, mask=None, dropout=None):
    "compute scaled dot product attention"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2,-1))/math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask==0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn,value), p_attn

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self,h,d_model,dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        self.d_k = d_model//h
        self.h = h
        self.linears = clones(nn.Linear(d_model,d_model),4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self,query,key,value,mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # do all linear proj in batch from d_model => h * d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h,self.d_k).transpose(1,2)
            for lin, x in zip(self.linears, (query,key,value))
        ]

        # apply attention on all projected vectors in batch.
        x, self.attn = attention(
            query,key,value,mask=mask,dropout=self.dropout
        )

        # concat using view and apply final linear.
        x = (
            x.transpose(1,2)
            .contiguous()
            .view(nbatches,-1,self.h*self.d_k)
        )
        del query
        del key
        del value

        return self.linears[-1](x)

In [15]:
torch.rand(4,3,512).view(4,-1,8,64).shape # into heads per seq

torch.Size([4, 3, 8, 64])

In [16]:
torch.rand(4,3,512).view(4,-1,8,64).transpose(1,2).shape # into seqs per head

torch.Size([4, 8, 3, 64])

FFN and Embeddings

In [17]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self,d_model,d_ff,dropout=0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.w_1 = nn.Linear(d_model,d_ff)
        self.w_2 = nn.Linear(d_ff,d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self,x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [18]:
ffn = PositionwiseFeedForward(512,2048)

In [19]:
ffn(torch.rand(3,4,512))

tensor([[[ 0.0118, -0.0520,  0.0460,  ...,  0.0097,  0.2788, -0.1766],
         [ 0.0502,  0.0106, -0.0035,  ..., -0.0663,  0.1567, -0.1496],
         [ 0.0162, -0.0596,  0.1255,  ..., -0.0254,  0.1744,  0.0534],
         [ 0.1888, -0.1108,  0.0816,  ...,  0.1911,  0.2350, -0.1343]],

        [[ 0.0274, -0.0031, -0.0575,  ...,  0.0848,  0.1802, -0.1993],
         [ 0.0591,  0.0117,  0.2198,  ..., -0.0434,  0.0128, -0.1241],
         [ 0.0835, -0.0556,  0.1465,  ...,  0.1510,  0.1317, -0.1677],
         [-0.0018, -0.0270, -0.0737,  ...,  0.0573,  0.0062, -0.1245]],

        [[-0.0258, -0.0746,  0.1187,  ..., -0.0866,  0.1113, -0.2158],
         [ 0.1626, -0.0341,  0.0792,  ...,  0.0794,  0.1311, -0.1301],
         [ 0.2096, -0.2094,  0.0165,  ..., -0.0553,  0.1145, -0.1533],
         [ 0.1490, -0.0736,  0.0971,  ..., -0.0034,  0.0821, -0.1988]]],
       grad_fn=<ViewBackward0>)

In [20]:
class Embeddings(nn.Module):

    def __init__(self,d_model,vocab):
        super(Embeddings,self).__init__()
        self.lut = nn.Embedding(vocab,d_model)
        self.d_model = d_model

    def forward(self,x):
        return self.lut(x) * math.sqrt(self.d_model)

In [21]:
class PositionalEncoding(nn.Module):

    def __init__(self,d_model,dropout,max_len=5000):
        super(PositionalEncoding,self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # compute the positional encodings once in log space
        pe = torch.zeros(max_len,d_model)
        position = torch.arange(0,max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0,d_model,2) * -(math.log(10000.0)/d_model)
        )
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer("pe",pe)

    def forward(self,x):
        x = x + self.pe[:,: x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [22]:
import altair as alt

RUN_EXAMPLES = True

def show_example(fn,args=[]):
    if __name__=="__main__" and RUN_EXAMPLES:
        return fn(*args)

def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )
show_example(example_positional)

alt.Chart(...)

Encoder-Decoder

In [23]:
class EncoderDecoder(nn.Module):

    def __init__(self,encoder,decoder,src_embed,tgt_embed,generator):
        super(EncoderDecoder,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self,src,tgt,src_mask,tgt_mask):
        return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask)
    
    def encode(self,src,src_mask):
        return self.encoder(self.src_embed(src),src_mask)

    def decode(self,memeory,src_mask,tgt,tgt_mask):
        return self.decoder(self.tgt_embed(tgt),memeory,src_mask,tgt_mask)

In [24]:
class Generator(nn.Module):

    def __init__(self,d_model,vocab):
        super(Generator,self).__init__()
        self.proj = nn.Linear(d_model,vocab)

    def forward(self,x):
        return log_softmax(self.proj(x),dim=-1)

In [25]:
def make_model(
        src_vocab, tgt_vocab, N=6, d_model=512, d_ff = 2048, h =8, dropout=0.1
):
    "Construct the model from hyperparameters"
    c = copy.deepcopy
    attn = MultiHeadAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model,dropout)

    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model,c(attn),c(ff),dropout),N),
        Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),
        nn.Sequential(Embeddings(d_model,src_vocab),c(position)),
        nn.Sequential(Embeddings(d_model,tgt_vocab),c(position)),
        Generator(d_model,tgt_vocab)
    )

    # initialize parameters with glorot / fan_avg
    for p in model.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
    return model


    

Model Training

In [26]:
def inference_test():
    test_model = make_model(11,11,2)
    test_model.eval()
    src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]])
    src_mask = torch.ones(1,1,10)

    memory = test_model.encode(src,src_mask)
    ys = torch.zeros(1,1).type_as(src)

    for i in range(9):
        out = test_model.decode(
            memory,src_mask,ys,subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = test_model.generator(
            out[:,-1]
        )
        _,next_word = torch.max(prob,dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1,1).type_as(src.data).fill_(next_word)],
            dim=1
        )
        print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()

show_example(run_tests)

Example Untrained Model Prediction: tensor([[0, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 5, 5, 5, 5, 5, 5, 5, 5]])
Example Untrained Model Prediction: tensor([[0, 7]])
Example Untrained Model Prediction: tensor([[0, 7, 1]])
Example Untrained Model Prediction: tensor([[0, 7, 1, 7]])
Example Untrained Model Prediction: tensor([[0, 7, 1, 7, 1]])
Example Untrained Model Prediction: tensor([[0, 7, 1, 7, 1, 7]])
Example Untrained Model Prediction: tensor([[0, 7, 1, 7, 1, 7, 1]])
Example Untrained Model Prediction: tensor([[0, 7, 1

In [27]:
src = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]])
pad = 1
src_mask = (src!=pad).unsqueeze(-2)
src_mask

tensor([[[False,  True,  True,  True,  True,  True,  True,  True,  True,  True]]])

In [28]:
# batches and masking
class Batch:

    def __init__(self,src,tgt=None,pad=2): # 2 = <blank>

        self.src = src
        self.src_mask = (src!=pad).unsqueeze(-2)

        if tgt is not None:
            self.tgt = tgt[:,:-1]
            self.tgt_y = tgt[:,1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()
        
    @staticmethod
    def make_std_mask(tgt,pad):
        "create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask


Training Loop

In [29]:
class Trainstep:

    step: int= 0
    accum_step : int = 0
    samples: int = 0
    tokens: int = 0

def run_epoch(
        data_iter,
        model,
        loss_compute,
        optimizer,
        scheduler,
        mode="train",
        accum_iter=1,
        train_state=Trainstep()
):
    "train a single epoch"
    start = time.time()
    total_tokens=0
    total_loss=0
    tokens=0
    n_accum=0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y,batch.ntokens)

        # loss_node = loss_node / accum_iter

        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step +=1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()
        total_loss+=loss
        total_tokens+=batch.ntokens
        tokens+= batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
    return total_loss / total_tokens, train_state

Regularization

In [37]:
class LabelSmoothing(nn.Module):

    def __init__(self,size,padding_idx,smoothing=0.0):
        super(LabelSmoothing,self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0-smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self,x:torch.Tensor,target):
        assert x.size(1)== self.size
        true_dist = x.data.clone()

        # fill every value with wrong idx smoothing 
        true_dist.fill_(self.smoothing / (self.size-2))
        # now correct idx with smoothing
        true_dist.scatter_(1, target.data.unsqueeze(1),self.confidence)
        # Zero out the PAD token distribution
        true_dist[:,self.padding_idx]=0
        # If the target itself is PAD, zero out the entire row
        mask = torch.nonzero(target.data == self.padding_idx)

        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(),0.0)
        self.true_dist = true_dist

        return self.criterion(x, true_dist.clone().detach())



In [38]:
def example_label_smoothing():
    crit = LabelSmoothing(5, 0, 0.4)
    predict = torch.FloatTensor(
        [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
        ]
    )
    crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect(color="Blue", opacity=1)
        .properties(height=200, width=200)
        .encode(
            alt.X("columns:O", title=None),
            alt.Y("rows:O", title=None),
            alt.Color(
                "target distribution:Q", scale=alt.Scale(scheme="viridis")
            ),
        )
        .interactive()
    )


show_example(example_label_smoothing)

alt.Chart(...)

A First Example

In [51]:
def data_gen(V, batch_size, nbatches):
    "generate random data for a src-tgt copy task"
    for i in range(nbatches):
        data = torch.randint(1,V, size=(batch_size,V))
        data[:,0] = 1
        src = data.requires_grad_(False).clone().detach()
        tgt = data.requires_grad_(False).clone().detach()
        yield Batch(src,tgt,pad=0)

In [52]:
vocab=3
batch_size=2
nbatches = 1
data_iter = data_gen(vocab,batch_size,nbatches)
for i,batch in enumerate(data_iter):
    print(batch.src_mask)
    print(batch.tgt_mask)

tensor([[[True, True, True]],

        [[True, True, True]]])
tensor([[[ True, False],
         [ True,  True]],

        [[ True, False],
         [ True,  True]]])


In [41]:
# Loss Computation
class SimpleLossCompute:

    def __init__(self,generator: Generator ,criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self,x,y,norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1,x.size(-1)),
                y.contiguous().view(-1)
            )
            /norm
        )
        return sloss.data * norm, sloss

Greedy Decoding

In [42]:
def greedy_decode(model: EncoderDecoder,src,src_mask,max_len,start_symbol):
    memory = model.encode(src,src_mask)
    ys = torch.zeros(1,1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(
            memory, src_mask, ys,
            subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:,-1])
        _, next_word = torch.max(prob,dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1,1).type_as(src.data).fill_(next_word)],
            dim=1
        )
        return ys

In [43]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

In [44]:
class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

In [45]:
V = 11
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
model = make_model(V,V, N=2)
optimizer = torch.optim.Adam(
    model.parameters(), lr=0.5, betas=(0.9,0.98), eps=1e-9
)
lr_scheduler = LambdaLR(
    optimizer=optimizer,
    lr_lambda= lambda step: rate(
        step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
    )
)
batch_size = 80
for epoch in range(20):
    model.train()
    run_epoch(
        data_gen(V, batch_size,20),
        model,
        SimpleLossCompute(model.generator, criterion),
        optimizer,
        lr_scheduler,
        mode="train"
    )
    model.eval()
    run_epoch(
        data_gen(V,batch_size,5),
        model,
        SimpleLossCompute(model.generator, criterion),
        DummyOptimizer(),
        DummyScheduler(),
        mode="eval"
    )[0]


Epoch Step:      1 | Accumulation Step:   2 | Loss:   3.25 | Tokens / Sec:  1321.5 | Learning Rate: 5.5e-06
Epoch Step:      1 | Accumulation Step:   2 | Loss:   2.06 | Tokens / Sec:  1641.2 | Learning Rate: 6.1e-05
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.74 | Tokens / Sec:  1656.7 | Learning Rate: 1.2e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.51 | Tokens / Sec:  1446.4 | Learning Rate: 1.7e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.07 | Tokens / Sec:  1492.0 | Learning Rate: 2.3e-04


KeyboardInterrupt: 